In [1]:
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json
import creds

In [2]:
username = creds.username
password = creds.password
appkey = creds.Dev_Appkey
# Get local dir for path to certs
certs_p = '/certs'

In [3]:
trading = betfairlightweight.APIClient(username=username,
                                      password=password,
                                      app_key=appkey,
                                       certs=certs_p)

In [4]:
trading.login()

<LoginResource>

In [6]:
# Grab all event type ids and print out
event_types = trading.betting.list_event_types()

sport_ids = pd.DataFrame({
    'Sport': [event_type_object.event_type.name for event_type_object in event_types],
    'ID': [event_type_object.event_type.id for event_type_object in event_types]
}).set_index('Sport').sort_index()

sport_ids

,ID
Sport,
American Football,6423
Athletics,3988
Australian Rules,61420
Badminton,627555
Baseball,7511
Basketball,7522
Beach Volleyball,2872194
Boxing,6
Cricket,4


In [8]:
# FIlter event type by name
horse_racing_filter = betfairlightweight.filters.market_filter(text_query='Horse Racing')

# Returns a list
horse_racing_event_type = trading.betting.list_event_types(
                            filter=horse_racing_filter)

# first element of list
horse_racing_event_type = horse_racing_event_type[0]

horse_racing_event_type_id = horse_racing_event_type.event_type.id
print(f'The event type id for horse racing is {horse_racing_event_type_id}')

The event type id for horse racing is 7


In [15]:
# Get a datetime object in a week and convert to string
datetime_in_a_week = (datetime.datetime.utcnow() + datetime.timedelta(weeks=1)).strftime("%Y-%m-%dT%TZ")

# Create a competition filter
competition_filter = betfairlightweight.filters.market_filter(
    event_type_ids=[1], # Soccer's event type id is 1
    market_start_time={
        'to': datetime_in_a_week
    })

# Get a list of competitions for soccer
competitions = trading.betting.list_competitions(
    filter=competition_filter
)

# Iterate over the competitions and create a dataframe of comps and ids
soccer_competitions = pd.DataFrame({
    'Competition': [competition_object.competition.name for competition_object in competitions],
    'ID': [competition_object.competition.id for competition_object in competitions]
})

In [16]:
# Get english premier league
soccer_competitions[soccer_competitions.Competition.str.contains('English Premier')]

,Competition,ID


In [17]:
soccer_competitions

,Competition,ID
0,Specials,2608550
1,Ecuadorian Serie B,3057583
2,Swedish Allsvenskan,129
3,Brazilian Cup,89219
4,Czech 3 Liga,892425
...,...,...
78,Irish Premier Division,12203971
79,Belgian First Division A,89979
80,Icelandic 1 Deild,12010570
81,UEFA Nations League,11984200


In [20]:
# Get upcoming events
# Define market filter
thoroughbreds_event_filter = betfairlightweight.filters.market_filter(
        event_type_ids=[horse_racing_event_type_id],
        market_countries=['AU'],
        market_start_time={
            'to': (datetime.datetime.utcnow() + datetime.timedelta(days=1)).strftime('%Y-%m-%dT%TZ')
        })

thoroughbreds_event_filter

{'eventTypeIds': ['7'],
 'marketCountries': ['AU'],
 'marketStartTime': {'to': '2021-07-30T03:15:04Z'}}

In [22]:
# Get list of all thoroughbred events as objects
aus_thoroughbred_events = trading.betting.list_events(
    filter=thoroughbreds_event_filter)

# Create dataframe with all the events by iterating over each event object
aus_thoroughbred_events_today = pd.DataFrame({
    'Event Name': [event_object.event.name for event_object in aus_thoroughbred_events],
    'Event ID': [event_object.event.id for event_object in aus_thoroughbred_events],
    'Event Venue': [event_object.event.venue for event_object in aus_thoroughbred_events],
    'Country Code': [event_object.event.country_code for event_object in aus_thoroughbred_events],
    'Time Zone': [event_object.event.time_zone for event_object in aus_thoroughbred_events],
    'Open Date': [event_object.event.open_date for event_object in aus_thoroughbred_events],
    'Market Count': [event_object.market_count for event_object in aus_thoroughbred_events]
})

aus_thoroughbred_events_today

,Event Name,Event ID,Event Venue,Country Code,Time Zone,Open Date,Market Count
0,Wyong (AUS) 29th Jul,30734917,Wyong,AU,Australia/Sydney,2021-07-29 02:25:00,13
1,Albion Park (AUS) 29th Jul,30735173,Albion Park,AU,Australia/Queensland,2021-07-29 01:31:00,10
2,Cairns (AUS) 29th Jul,30734902,Cairns,AU,Australia/Queensland,2021-07-29 02:43:00,19
3,Bendigo (AUS) 29th Jul,30735392,Bendigo,AU,Australia/Sydney,2021-07-29 07:17:00,18
4,Stawell (AUS) 29th Jul,30735393,Stawell,AU,Australia/Sydney,2021-07-29 03:28:00,12
5,Gawler (AUS) 29th Jul,30735262,Gawler,AU,Australia/Adelaide,2021-07-29 02:35:00,18
6,Northam (AUS) 29th Jul,30735034,Northam,AU,Australia/Perth,2021-07-29 05:13:00,21
7,Penrith (AUS) 29th Jul,30735018,Penrith,AU,Australia/Sydney,2021-07-29 08:14:00,18
8,Tamworth (AUS) 29th Jul,30735019,Tamworth,AU,Australia/Sydney,2021-07-29 03:48:00,16


In [23]:
# GEt market type
# Define market fileter
market_types_filter = betfairlightweight.filters.market_filter(event_ids=['30734917'])

# Request market types
market_types = trading.betting.list_market_types(
    filter=market_types_filter)

# Create a df of market types
market_types_mooney_valley = pd.DataFrame({
    'Market Type': [market_type_object.market_type for market_type_object in market_types],
})

market_types_mooney_valley

,Market Type
0,WIN
1,PLACE


In [24]:
# Market catalogues
market_catalogue_filter = betfairlightweight.filters.market_filter(event_ids=['30734917'])

market_catalogues = trading.betting.list_market_catalogue(
    filter=market_catalogue_filter,
    max_results='100',
    sort='FIRST_TO_START')

# Create DF from each market catalogue
market_types_mooney_valley = pd.DataFrame({
    'Market Name': [market_cat_object.market_name for market_cat_object in market_catalogues],
    'Market ID': [market_cat_object.market_id for market_cat_object in market_catalogues],
    'Total Matched': [market_cat_object.total_matched for market_cat_object in market_catalogues]
})

market_types_mooney_valley

,Market Name,Market ID,Total Matched
0,R3 1600m Mdn,1.185728151,11103.095820
1,To Be Placed,1.185728150,493.842750
2,R4 1350m 3yo,1.185728153,977.940840
3,To Be Placed,1.185728152,47.816820
4,R5 1200m CL1,1.185728155,553.141650
5,To Be Placed,1.185728154,55.030890
6,R6 1100m Hcap,1.185728157,2545.528035
7,To Be Placed,1.185728156,262.274880
8,R7 1600m Hcap,1.185728159,1402.211250
9,To Be Placed,1.185728158,56.050680


In [28]:
def process_runner_books(runner_books):
    ''' 
    Function to process runner books and return a df with best back/lay prices + volume
    Params: runner books
    '''
    best_back_prices = [runner_book.ex.available_to_back[0].price if runner_book.ex.available_to_back else 1.01 for runner_book in runner_books]
    best_back_sizes = [runner_book.ex.available_to_back[0].size if runner_book.ex.available_to_back else 1.01 for runner_book in runner_books]
    best_lay_prices = [runner_book.ex.available_to_lay[0].price if runner_book.ex.available_to_lay else 10000.0 for runner_book in runner_books]
    best_lay_sizes = [runner_book.ex.available_to_lay[0].size if runner_book.ex.available_to_lay else 1.01 for runner_book in runner_books]
    
    selection_ids = [runner_book.selection_id for runner_book in runner_books]
    last_prices_traded = [runner_book.last_price_traded for runner_book in runner_books]
    total_matched = [runner_book.total_matched for runner_book in runner_books]
    statuses = [runner_book.status for runner_book in runner_books]
    scratching_datetimes = [runner_book.removal_date for runner_book in runner_books]
    adjustment_factors = [runner_book.adjustment_factor for runner_book in runner_books]
    
    df = pd.DataFrame({
        'Selection ID': selection_ids,
        'Best Back Price': best_back_prices,
        'Best Back Size': best_back_sizes,
        'Best Lay Price': best_lay_prices,
        'Best Lay Size': best_lay_sizes,
        'Last Price Traded': last_prices_traded,
        'Total Matched': total_matched,
        'Status': statuses,
        'Removal Date': scratching_datetimes,
        'Adjustment Factor': adjustment_factors
    })
    return df

In [29]:
# Create price filter. Get all traded and offer data
price_filter = betfairlightweight.filters.price_projection(
    price_data=['EX_BEST_OFFERS'])

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=['1.185728151'],
    price_projection=price_filter)

# Grab first market book from the returned list
market_book = market_books[0]

runners_df = process_runner_books(market_book.runners)

runners_df

,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
0,4335393,1.01,1.01,10000.0,1.01,None,None,REMOVED,2021-07-28 09:03:39,0.669
1,40346640,1.01,1.01,10000.0,1.01,None,None,REMOVED,2021-07-28 21:17:10,9.063
2,40198208,1.01,1.01,10000.0,1.01,None,None,LOSER,NaT,5.385
3,40198470,1.01,1.01,10000.0,1.01,None,None,WINNER,NaT,20.877
4,40346638,1.01,1.01,10000.0,1.01,None,None,LOSER,NaT,13.812
5,40346639,1.01,1.01,10000.0,1.01,None,None,LOSER,NaT,30.125
6,40031905,1.01,1.01,10000.0,1.01,None,None,LOSER,NaT,16.155
7,38589943,1.01,1.01,10000.0,1.01,None,None,LOSER,NaT,4.708
8,38946178,1.01,1.01,10000.0,1.01,None,None,LOSER,NaT,7.843
9,40346641,1.01,1.01,10000.0,1.01,None,None,LOSER,NaT,1.095


In [30]:
# OVERBOOK WORKFLOW

# Placing orders DONT ACTUALLY PLACE THEM (bet on fav strategy)
# Get favourites price and selection_id
fav_selection_id = runners_df.loc[runners_df['Best Back Price'].idxmin(), 'Selection ID']
fav_price = runners_df.loc[runners_df['Best Back Price'].idxmin(), 'Best Back Price']


In [32]:
# Define limit order filter
limit_order_filter = betfairlightweight.filters.limit_order(
    size=0,
    price=0,
    persistence_type='LAPSE')

# Define instructions filter
instructions_filter = betfairlightweight.filters.place_instruction(
    selection_id=str(fav_selection_id),
    order_type='LIMIT',
    side='BACK',
    limit_order=limit_order_filter)

instructions_filter

{'orderType': 'LIMIT',
 'selectionId': '4335393',
 'side': 'BACK',
 'limitOrder': {'price': 0, 'persistenceType': 'LAPSE', 'size': 0}}

In [ ]:
# Place order # DONT HIT THIS
order = trading.betting.place_orders(
    market_id='1.185728151', # Market id we have before
    customer_strategy_ref='back_the_fav',
    instructions=[instructions_filter] # Must be a list)

In [ ]:
order.__dict__

In [34]:
# Get current orders
trading.betting.list_current_orders(customer_strategy_refs=['back_the_fav']).__dict__

{'streaming_unique_id': None,
 'streaming_update': None,
 'streaming_snap': False,
 'publish_time': None,
 'elapsed_time': 0.40309977531433105,
 '_datetime_created': datetime.datetime(2021, 7, 29, 3, 54, 39, 713658),
 '_datetime_updated': datetime.datetime(2021, 7, 29, 3, 54, 39, 713658),
 '_data': {'currentOrders': [], 'moreAvailable': False},
 'more_available': False,
 'orders': [],
 'matches': []}

In [ ]:
# Cancelling orders
cancelled_order = trading.betting.cancel_orders(market_id='1.185728151')
# Create a DF to view the instruction report
pd.Series(cancelled_order.cancel_instruction_reports[0].__dict__).to_frame().T